In [98]:
import pandas as pd

In [203]:
dem = pd.read_pickle('pts_inc.pkl')
cc = dem[['f.eid']].copy()
cutoff = pd.read_pickle('./Final/cutoff.pkl')
dx = pd.read_pickle('./Raw/dx.pkl').rename({'dx_date':'Date'},axis=1)
dx = dx.loc[dx['Date'] != '2037-07-07']
op = pd.read_pickle('./Raw/operations.pkl')
self_dx = pd.read_pickle('./Raw/self_report_illness.pkl')
self_op = pd.read_pickle('./Raw/self_report_operations.pkl')

In [230]:
##### AFib

d1 = dx.loc[dx['icd10'].str.contains('I48', na=False)][['f.eid','Date']]
d2 = op.loc[op['Code'].str.contains('K571|K621|K622|K623', na=False)][['f.eid','Date']]
d3 = self_dx.loc[self_dx['Code'].isin([1471,1483])][['f.eid','Date']]
d4 = self_op.loc[self_op['Code'] == 1524][['f.eid','Date']]

temp = pd.concat([d1,d2,d3,d4])
temp.loc[temp['Date'].isna(), 'Date'] = '1900-01-01'
temp['Date'] = pd.to_datetime(temp['Date']).dt.date
temp = temp.sort_values('Date').drop_duplicates('f.eid')

temp['afib_status'] = 1
temp['afib_date'] = temp['Date'].copy()
temp = temp.drop('Date',axis=1)

temp = cc.merge(temp, on='f.eid', how='left')
temp = temp.merge(cutoff, on='f.eid', how='left')
temp.loc[temp['afib_date'].isna(), 'afib_date'] = temp['Cutoff']
temp['afib_date'] = pd.to_datetime(temp['afib_date']).dt.date
temp = temp.drop('Cutoff',axis=1)

temp.loc[temp['afib_status'].isna(), 'afib_status'] = 0
temp.to_pickle('./Final/Cases/afib.pkl')

In [219]:
##### CAD

d1 = dx.loc[(dx['source'] == 'inpatient') & (dx['icd10'].str.contains('I21|I220|I221|I228|I229|I230|I231|I232|I233|I235|I236|I238|I24|I251|I252|I255|I256|I258|I259', na=False))][['f.eid','Date']]
d1a = dx.loc[(dx['source'] == 'gp') & (dx['icd10'].str.contains('I21|I220|I221|I228|I229|I230|I231|I232|I233|I235|I236|I238|I24|I251|I252|I255|I256|I258|I259', na=False))][['f.eid','Date']]
d1b = dx.loc[(dx['source'] == 'death') & (dx['icd10'].str.contains('I214|I219|I252', na=False))][['f.eid','Date']]
d2 = op.loc[op['Code'].str.contains('K401|K402|K403|K404|K412|K413|K414|K451|K75', na=False)][['f.eid','Date']]
d2a = op.loc[op['Code'].isin([3041,3042,3043])][['f.eid','Date']]
d3 = self_dx.loc[self_dx['Code'] == 1075][['f.eid','Date']]
d4 = self_op.loc[self_op['Code'].isin([1070,1095])][['f.eid','Date']]

##### Exclude angina

d1ex = dx.loc[dx['icd10'].str.contains('I20', na=False)]['f.eid'].to_list()
d3ex = self_dx.loc[self_dx['Code'] == 1074]['f.eid'].to_list()

temp = pd.concat([d1,d1a,d1b,d2,d2a,d3,d4])
temp.loc[temp['Date'].isna(), 'Date'] = '1900-01-01'
temp['Date'] = pd.to_datetime(temp['Date']).dt.date
temp = temp.sort_values('Date').drop_duplicates('f.eid')

temp['cad_status'] = 1
temp['cad_date'] = temp['Date'].copy()
temp = temp.drop('Date',axis=1)

temp = cc.merge(temp, on='f.eid', how='left')
temp = temp.merge(cutoff, on='f.eid', how='left')
temp.loc[temp['cad_date'].isna(), 'cad_date'] = temp['Cutoff']
temp['cad_date'] = pd.to_datetime(temp['cad_date']).dt.date
temp = temp.drop('Cutoff',axis=1)

temp.loc[temp['cad_status'].isna(), 'cad_status'] = 0
temp.loc[(temp['cad_status'] == 0) & (temp['f.eid'].isin(list(set(d1ex+d3ex)))), 'cad_status'] = -1
temp.to_pickle('./Final/Cases/cad.pkl')

In [ ]:
##### Celiac

d1 = dx.loc[dx['icd10'].str.contains('K900', na=False)][['f.eid','Date']]
d3 = self_dx.loc[self_dx['Code'] == 1456][['f.eid','Date']]

print(len(d1),len(d3))

temp = pd.concat([d1,d3])
temp.loc[temp['Date'].isna(), 'Date'] = '1900-01-01'
temp['Date'] = pd.to_datetime(temp['Date']).dt.date
temp = temp.sort_values('Date').drop_duplicates('f.eid')

temp['celiac_status'] = 1
temp['celiac_date'] = temp['Date'].copy()
temp = temp.drop('Date',axis=1)

temp = cc.merge(temp, on='f.eid', how='left')
temp = temp.merge(cutoff, on='f.eid', how='left')
temp.loc[temp['celiac_date'].isna(), 'celiac_date'] = temp['Cutoff']
temp['celiac_date'] = pd.to_datetime(temp['celiac_date']).dt.date
temp = temp.drop('Cutoff',axis=1)

temp.loc[temp['celiac_status'].isna(), 'celiac_status'] = 0
temp.to_pickle('./Final/Cases/celiac.pkl')

In [234]:
##### Gallstone

d1 = dx.loc[dx['icd10'].str.contains('K80', na=False)][['f.eid','Date']]
d3 = self_dx.loc[self_dx['Code'] == 1162][['f.eid','Date']]

temp = pd.concat([d1,d3])
temp.loc[temp['Date'].isna(), 'Date'] = '1900-01-01'
temp['Date'] = pd.to_datetime(temp['Date']).dt.date
temp = temp.sort_values('Date').drop_duplicates('f.eid')

temp['gallstone_status'] = 1
temp['gallstone_date'] = temp['Date'].copy()
temp = temp.drop('Date',axis=1)

temp = cc.merge(temp, on='f.eid', how='left')
temp = temp.merge(cutoff, on='f.eid', how='left')
temp.loc[temp['gallstone_date'].isna(), 'gallstone_date'] = temp['Cutoff']
temp['gallstone_date'] = pd.to_datetime(temp['gallstone_date']).dt.date
temp = temp.drop('Cutoff',axis=1)

temp.loc[temp['gallstone_status'].isna(), 'gallstone_status'] = 0
temp.to_pickle('./Final/Cases/gallstone.pkl')

In [ ]:
##### Polyp

d1 = dx.loc[dx['icd10'].str.contains('J33', na=False)][['f.eid','Date']]
d2 = op.loc[op['Code'].str.contains('E081', na=False)][['f.eid','Date']]
d2a = op.loc[op['Code'] == 216][['f.eid','Date']]
d3 = self_dx.loc[self_dx['Code'] == 1417][['f.eid','Date']]
d4 = self_op.loc[self_op['Code'] == 1559][['f.eid','Date']]

print(len(d1),len(d2),len(d2a),len(d3),len(d4))

temp = pd.concat([d1,d2,d2a,d3,d4])
temp.loc[temp['Date'].isna(), 'Date'] = '1900-01-01'
temp['Date'] = pd.to_datetime(temp['Date']).dt.date
temp = temp.sort_values('Date').drop_duplicates('f.eid')

temp['polyp_status'] = 1
temp['polyp_date'] = temp['Date'].copy()
temp = temp.drop('Date',axis=1)

temp = cc.merge(temp, on='f.eid', how='left')
temp = temp.merge(cutoff, on='f.eid', how='left')
temp.loc[temp['polyp_date'].isna(), 'polyp_date'] = temp['Cutoff']
temp['polyp_date'] = pd.to_datetime(temp['polyp_date']).dt.date
temp = temp.drop('Cutoff',axis=1)

temp.loc[temp['polyp_status'].isna(), 'polyp_status'] = 0
temp.to_pickle('./Final/Cases/polyp.pkl')

In [264]:
##### T2D

d1 = dx.loc[dx['icd10'].str.contains('E11', na=False)][['f.eid','Date']]
d3 = self_dx.loc[self_dx['Code'] == 1223][['f.eid','Date']]

med = pd.read_pickle('./Final/med_atc.pkl')[['f.eid','instance','A10B']]
med = med.loc[med['A10B'] == 1]
visits = pd.read_pickle('/sc/arion/projects/GENECAD/Robert/NAFLD/Raw/visit_dates.pkl')
visits = visits.rename({'visit':'instance'},axis=1)
med['instance'] = med['instance'].astype(int)
visits['instance'] = visits['instance'].astype(int)
med = med.merge(visits)[['f.eid','date']].rename({'date':'Date'},axis=1)

##### Exclude T1D from all participants

d1ex = dx.loc[dx['icd10'].str.contains('E10', na=False)]['f.eid'].to_list()
d3ex = self_dx.loc[self_dx['Code'] == 1222]['f.eid'].to_list()

##### Exclude unspecified diabetes from controls

d5ex = self_dx.loc[self_dx['Code'] == 1220]['f.eid'].to_list()

temp = pd.concat([d1,d3,med])
temp.loc[temp['Date'].isna(), 'Date'] = '1900-01-01'
temp['Date'] = pd.to_datetime(temp['Date']).dt.date
temp = temp.sort_values('Date').drop_duplicates('f.eid')

temp['t2d_status'] = 1
temp['t2d_date'] = temp['Date'].copy()
temp = temp.drop('Date',axis=1)

temp = cc.merge(temp, on='f.eid', how='left')
temp = temp.merge(cutoff, on='f.eid', how='left')
temp.loc[temp['t2d_date'].isna(), 't2d_date'] = temp['Cutoff']
temp['t2d_date'] = pd.to_datetime(temp['t2d_date']).dt.date
temp = temp.drop('Cutoff',axis=1)

temp.loc[temp['t2d_status'].isna(), 't2d_status'] = 0
temp.loc[temp['f.eid'].isin(list(set(d1ex+d3ex))), 't2d_status'] = -1
temp.loc[(temp['t2d_status'] == 0) & (temp['f.eid'].isin(list(set(d5ex)))), 't2d_status'] = -1
temp.to_pickle('./Final/Cases/t2d.pkl')

In [238]:
##### Varicose

d1 = dx.loc[dx['icd10'].str.contains('I83', na=False)][['f.eid','Date']]
d2 = op.loc[op['Code'].str.contains('L84|L85|L86|L87|L88', na=False)][['f.eid','Date']]
d2a = op.loc[op['Code'].isin([8931, 894, 8941])][['f.eid','Date']]
d3 = self_dx.loc[self_dx['Code'] == 1494][['f.eid','Date']]
d4 = self_op.loc[self_op['Code'] == 1479][['f.eid','Date']]

print(len(d1),len(d2),len(d2a),len(d3),len(d4))

temp = pd.concat([d1,d2,d2a,d3,d4])
temp.loc[temp['Date'].isna(), 'Date'] = '1900-01-01'
temp['Date'] = pd.to_datetime(temp['Date']).dt.date
temp = temp.sort_values('Date').drop_duplicates('f.eid')

temp['varicose_status'] = 1
temp['varicose_date'] = temp['Date'].copy()
temp = temp.drop('Date',axis=1)

temp = cc.merge(temp, on='f.eid', how='left')
temp = temp.merge(cutoff, on='f.eid', how='left')
temp.loc[temp['varicose_date'].isna(), 'varicose_date'] = temp['Cutoff']
temp['varicose_date'] = pd.to_datetime(temp['varicose_date']).dt.date
temp = temp.drop('Cutoff',axis=1)

temp.loc[temp['varicose_status'].isna(), 'varicose_status'] = 0
temp.to_pickle('./Final/Cases/varicose.pkl')

print(temp['varicose_status'].value_counts())

In [225]:
##### VTE

d1 = dx.loc[dx['icd10'].str.contains('I801|I802|I822|I824|I825|I82A|I82B|I82C|I26|O223|O871|O882', na=False)][['f.eid','Date']]
d2 = op.loc[op['Code'].str.contains('L791|L901|L902', na=False)][['f.eid','Date']]
d3 = self_dx.loc[self_dx['Code'].isin([1093,1094])][['f.eid','Date']]

##### Exclude SVT

d1ex = dx.loc[dx['icd10'].str.contains('I800|I803|I808|I809|O222|O870|I81|I820|I821|I823|I826|I827|I828|I829', na=False)]['f.eid'].to_list()
d3ex = self_dx.loc[self_dx['Code'] == 1068]['f.eid'].to_list()

temp = pd.concat([d1,d2,d3])
temp.loc[temp['Date'].isna(), 'Date'] = '1900-01-01'
temp['Date'] = pd.to_datetime(temp['Date']).dt.date
temp = temp.sort_values('Date').drop_duplicates('f.eid')

temp['vte_status'] = 1
temp['vte_date'] = temp['Date'].copy()
temp = temp.drop('Date',axis=1)

temp = cc.merge(temp, on='f.eid', how='left')
temp = temp.merge(cutoff, on='f.eid', how='left')
temp.loc[temp['vte_date'].isna(), 'vte_date'] = temp['Cutoff']
temp['vte_date'] = pd.to_datetime(temp['vte_date']).dt.date
temp = temp.drop('Cutoff',axis=1)

temp.loc[temp['vte_status'].isna(), 'vte_status'] = 0
temp.loc[(temp['vte_status'] == 0) & (temp['f.eid'].isin(list(set(d1ex+d3ex)))), 'vte_status'] = -1
temp.to_pickle('./Final/Cases/vte.pkl')